In [ ]:
!pip install torchtext==0.9.1

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext import data
from torchtext import datasets

from collections import Counter
from torchtext.vocab import vocab

import numpy as np
import random


In [ ]:
import torchtext
from torchtext.vocab import Vectors
import torch



In [ ]:
# import gensim.downloader as api
# wv = api.load('glove-twitter-50')

# for index, word in enumerate(wv.index_to_key):
#     if index == 10:
#         break
#     print(f"word #{index}/{len(wv.index_to_key)} is {word}")

In [ ]:
df_train = pd.read_csv("data/Twitter/hate_twitter/hate_train.csv")
df_test = pd.read_csv("data/Twitter/hate_twitter/hate_test.csv")
df_val = pd.read_csv("data/Twitter/hate_twitter/hate_val.csv")

print(df_train.shape)
df_train.head()

In [ ]:
print(df_train.shape)
df_train = df_train[df_train['clean_tweet'].notna()]
print(df_train.shape)
print(df_test.shape)
df_test = df_test[df_test['clean_tweet'].notna()]
print(df_test.shape)
print(df_val.shape)
df_val = df_val[df_val['clean_tweet'].notna()]
print(df_val.shape)

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = dtokenizer_language = 'english')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
with open('output_train.txt', 'w') as f:
    f.write(df_train['clean_tweet'].str.cat(sep='\n'))

with open('labels_train.txt', 'w') as f:
    f.write(df_train['label'].apply(str).str.cat(sep='\n'))
    
with open('output_test.txt', 'w') as f:
    f.write(df_test['clean_tweet'].str.cat(sep='\n'))
with open('labels_test.txt', 'w') as f:
    f.write(df_test['label'].apply(str).str.cat(sep='\n'))

with open('output_val.txt', 'w') as f:
    f.write(df_val['clean_tweet'].str.cat(sep='\n'))
with open('labels_val.txt', 'w') as f:
    f.write(df_val['label'].apply(str).str.cat(sep='\n'))
    
with open('output_train.txt', 'r') as f:
    tweets_train = f.read()
with open('labels_train.txt', 'r') as f:
    labels_train = f.read()

with open('output_test.txt', 'r') as f:
    tweets_test = f.read()
with open('labels_test.txt', 'r') as f:
    labels_test = f.read()
    
with open('output_val.txt', 'r') as f:
    tweets_val = f.read()
with open('labels_val.txt', 'r') as f:
    labels_val = f.read()

In [3]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

tokenizer = get_tokenizer('basic_english')
counter = Counter()
for text in tweets_int_train:
    for word in text:
        counter.update(tokenizer(word))
vocab_words = vocab(counter, min_freq=5)

ImportError: cannot import name 'vocab' from 'torchtext.vocab' (/Users/sophiamlawer/opt/anaconda3/lib/python3.8/site-packages/torchtext/vocab.py)

In [ ]:
default_index = -1
vocab_words.set_default_index(default_index)

In [2]:
from torchtext import vocab
from torchtext.vocab import GloVe

glove = GloVe(name="twitter.27B", dim=50)

# glove = vocab.GloVe('6B',cache=VECTORS_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab_words.get_itos())

.vector_cache/glove.twitter.27B.zip:  81%|████████  | 1.23G/1.52G [06:29<01:33, 3.15MB/s]   


OSError: [Errno 28] No space left on device

In [ ]:
TEXT.build_vocab(tweets_int_train, max_size=100000) 
LABEL.build_vocab(tweets_int_train) 

In [ ]:
print({k: TEXT.vocab.stoi[k] for k in list(TEXT.vocab.stoi)[:15]}) 
print(f"Most common 15 words in the vocab are: {TEXT.vocab.freqs.most_common(15)}")

In [ ]:
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = LabelField(tensor_type=torch.FloatTensor)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
BATCH_SIZE = 32
EMBEDDING_SIZE = 650
MAX_VOCAB_SIZE = 50000

In [ ]:
embed = loadEmbeddingMatrix()

In [ ]:
def loadEmbeddingMatrix():
        #load different embedding file from Kaggle depending on which embedding 
        #matrix we are going to experiment with
        EMBEDDING_FILE='../input/glove-twitter/glove.twitter.27B.25d.txt'
        embed_size = 25

        embeddings_index = dict()
        #Transfer the embedding weights into a dictionary by iterating through every line of the file.
        f = open(EMBEDDING_FILE)
        for line in f:
            #split up line into an indexed array
            values = line.split()
            #first index is word
            word = values[0]
            #store the rest of the values in the array as a new array
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs #50 dimensions
        f.close()
        print('Loaded %s word vectors.' % len(embeddings_index))

            
        gc.collect()
        #We get the mean and standard deviation of the embedding weights so that we could maintain the 
        #same statistics for the rest of our own random generated weights. 
        all_embs = np.stack(list(embeddings_index.values()))
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        #We are going to set the embedding size to the pretrained dimension as we are replicating it.
        #the size will be Number of Words in Vocab X Embedding Size
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        #With the newly created embedding matrix, we'll fill it up with the words that we have in both 
        #our own dictionary and loaded pretrained embedding. 
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            #then we see if this word is in glove's dictionary, if yes, get the corresponding weights
            embedding_vector = embeddings_index.get(word)
            #and store inside the embedding matrix that we will train later on.
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        #finally, return the embedding matrix
        return embedding_matrix

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == 'cpu':
    VECTORS_CACHE_DIR = '/Users/sophiamlawer/.vector_cache'
    # Please change above to your cache
else:
    VECTORS_CACHE_DIR = './.vector_cache'
    # This is the default cache on Colab. Caching may not work
    # as expected on Colab.
    
from itertools import combinations
from torchtext.vocab import GloVe

# glove = GloVe(name='6B')

In [ ]:
from torchtext.vocab import vocab
unk_token = '<unk>'
counter_words = Counter()
for (text) in tweets_split:
    counter_words.update(text)
# print(counter_words)
vocab_words = vocab(counter_words, specials=[unk_token]) 
vocab_words

In [ ]:
from collections import Counter
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")

def split_tweet(df):
    tweets_split = df.split('\n')
    all_text2 = ' '.join(tweets_split)
    # create a list of words
    words = all_text2.split()
    # Count all the words using Counter Method
    count_words = Counter(words)

    total_words = len(words)
    sorted_words = count_words.most_common(total_words)
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
    return tweets_split, vocab_to_int

def tweets_int(tweets_split):
    tweet_int =[]
    for tweet in tweets_split:
        tweet_int.append(tweet)
    return tweet_int

In [ ]:
tweets_split_train, _ = split_tweet(tweets_train)
print ('Number of tweets :', len(tweets_split_train))
tweets_split_test, _ = split_tweet(tweets_test) 
tweets_split_val, _ = split_tweet(tweets_val) 

tweets_int_train = tweets_int(tweets_split_train)
tweets_int_test = tweets_int(tweets_split_test) 
tweets_int_val = tweets_int(tweets_split_val) 


In [1]:
tweets_split_train[0]

NameError: name 'tweets_split_train' is not defined

In [ ]:
from torchtext import vocab

glove = vocab.GloVe('6B',cache=VECTORS_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab_words.get_itos())

In [ ]:
from collections import Counter
all_text2 = ' '.join(tweets_split)
# create a list of words
words = all_text2.split()
# Count all the words using Counter Method
count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [ ]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
vocab_to_int

In [ ]:
tweets_int = []
for tweet in tweets_split:
    r = [vocab_to_int[w] for w in tweet.split()]
    tweets_int.append(r)
print (tweets_int[0:3])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
tweets_len = [len(x) for x in tweets_int]
pd.Series(tweets_len).hist()
plt.show()
pd.Series(tweets_len).describe()

In [ ]:
def pad_features(tweets_int, seq_length):
    ''' 
    Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(tweets_int), seq_length), dtype = int)
    
    for i, tweet in enumerate(tweets_int):
        tweet_len = len(tweet)
        
        if tweet_len <= seq_length:
            zeroes = list(np.zeros(seq_length-tweet_len))
            new = zeroes+tweet
        elif tweet_len > seq_length:
            new = tweet[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features

In [ ]:
df2 = pad_features(tweets_int, 10)
df2

In [ ]:
labels = labels.split('\n')
encoded_labels = [1 if label =='1' else 0 for label in labels]


In [ ]:
len(encoded_labels)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

encoded_labels = np.array(encoded_labels)

train_data = TensorDataset(torch.from_numpy(df2), torch.from_numpy(encoded_labels))

batch_size = 50
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)


In [ ]:
train_data = list(preprocess_data_seq(train_0))
valid_data = list(preprocess_data_seq(valid_0))
test_data = list(preprocess_data_seq(test_0))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, 
                              collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE,
                              shuffle=False, 
                              collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

In [ ]:
import torch
import torch.nn as nn


class LSTM(nn.Module):
    """ Container module with an linear encoder/embedding, an RNN module, and a linear decoder.
    """

    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, glove, dropout=0,freeze_glove = False):
        ''' Initialize model parameters corresponding to ---
            - embedding layer
            - linear layer to map from hidden vector to the vocabulary
            - optionally, dropout layers.  Dropout layers can be placed after 
              the embedding layer or/and after the RNN layer. Dropout within
              an RNN is only applied when there are two or more num_layers.
            - optionally, initialize the model parameters.
            
            The arguments are:
            
            vocab_size: size of vocabulary
            embedding_dim: size of an embedding vector
            hidden_dim: size of hidden/state vector in RNN
            num_layers: number of layers in RNN
            dropout: dropout probability.
            
        '''
        super(LSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.glove = glove_
        self.vocab_size = vocab_size
        self.embed_size = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.freeze_glove = freeze_glove
        self.word_embeddings = torch.nn.Embedding.from_pretrained(self.glove, self.freeze_glove)
        self.rnn = nn.LSTM(self.embed_size, self.hidden_dim, self.n_layers, batch_first=True)
        self.hidden2tag = nn.Linear(self.hidden_dim, 1)
        

    def forward(self, input, hidden0):
        ''' 
        Run forward propagation for a given minibatch of inputs using
        hidden0 as the initial hidden state.

        In LSTMs hidden0 = (h_0, c_0). 

        The output of the RNN includes the hidden vector hiddenn = (h_n, c_n).
        Return this as well so that it can be used to initialize the next
        batch.
        
        Unlike previous homework sets do not apply softmax or logsoftmax here, since we'll use
        the more efficient CrossEntropyLoss.  See 
        https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html.
        '''
        batch_size = input.size(0)
        embeds = self.word_embeddings(input)
        print(hidden0[0].shape)
        lstm_out, hidden_new = self.rnn(embeds, hidden0)
#         lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        tag_space = self.hidden2tag(lstm_out)
        print(tag_space.shape)
#       sig_out = tag_space.view(batch_size, -1)
        tag_space = tag_space[:, -1]
        print(tag_space.shape)

        return tag_space, hidden_new


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
#         if (train_on_gpu):
#             hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
#                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
#         else:
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

    
model = LSTM(vocab_size, embedding_dim, hidden_dim, n_layers, glove_=glove_vector, freeze_glove=True)
print(model)

In [ ]:
# loss and optimization functions
lr=0.001

criterion =nn.BCELoss()
loss_function = torch.nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing
counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
# if(train_on_gpu):
#     model.cuda()

model.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = model.init_hidden(batch_size)
    
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

#         if(train_on_gpu):
#             inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.LongTensor)
        output, h = model(inputs, h)

        # calculate the loss and perform backprop
        print(output.squeeze().shape, labels.shape)
        loss = loss_function(output.squeeze(), labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

#                 if(train_on_gpu):
#                     inputs, labels = inputs.cuda(), labels.cuda()

                inputs = inputs.type(torch.LongTensor)
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())

            model.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
loss_function = nn.CrossEntropyLoss()

def train_an_epoch(dataloader, model):
    
    ### WRITE YOUR CODE BELOW
    model.train() # Sets the module in training mode.
    log_interval = 500
    hidden = model.init_hidden(batch_size)
    for idx, (text,label) in enumerate(dataloader):
        model.zero_grad() #initialize to 0 gradient
        out, hidden = model(text, hidden) #picking forward method
        loss = loss_function(out.view(-1, vocab_size), label.view(-1)) #calcs the loss
        loss.backward() #backpropogation
        optimizer.step() #changing the gradient descent
        if idx % log_interval == 0 and idx > 0:
            print(f'At iteration {idx} the loss is {loss:.3f}.')
            
        

In [ ]:
def get_accuracy(dataloader, model):
    model.eval()
    with torch.no_grad():    
        total_acc, total_count = 0, 0
        for idx, (label, word_idxs) in enumerate(dataloader):
            log_probs = model(word_idxs)
            total_acc += (log_probs.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

EPOCHS = 3 # epoch
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

accuracies=[]
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train_an_epoch(train_loader, model_glove_fro)
    accuracy = get_accuracy(train_loader, model_glove_fro)
    accuracies.append(accuracy)
    time_taken = time.time() - epoch_start_time
    print(f'Epoch: {epoch}, time taken: {time_taken:.1f}s, validation accuracy: {accuracy:.3f}.')
    
plt.plot(range(1, EPOCHS+1), accuracies)


In [ ]:
!pip install torchtext==0.2.3

In [ ]:
%%writefile load_data.py

import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext.legacy import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

def load_dataset(test_sen=None):

    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fi_length which
                 will pad each sequence to have a fix length of 200.
                 
    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.
                  
    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
    
    """
    
    tokenize = lambda x: x.split()
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    LABEL = torchtext.data.LabelField(tensor_type=torch.FloatTensor)
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors
    print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
    print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
    print ("Label Length: " + str(len(LABEL.vocab)))

    train_data, valid_data = train_data.split() # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

    '''Alternatively we can also use the default configurations'''
    # train_iter, test_iter = datasets.IMDB.iters(batch_size=32)

    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter

In [ ]:
import load_data
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_data.load_dataset()

In [ ]:
import pandas as pd # data handling
import numpy as np # base of all
import matplotlib.pyplot as plt # plotting
import seaborn as sns  # advance plotting
# from wordcloud import WordCloud # to see the words as image
import torch # PyTorch for building Networks
from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator, Iterator
from torchtext import vocab
from sklearn.model_selection import train_test_split # split the data into training and testing
from sklearn.metrics import accuracy_score # accuracy metric
from nltk import word_tokenize # very popular Text processing Library
import random # to perform randomisation of tasks
from tqdm.notebook import tqdm # for a continuous progress bar style
import time # time module 
import os # import operating system

In [ ]:
OUT_PATH = "data/Twitter/hate_twitter/"
text_field = Field(tokenize=word_tokenize)
label_field = LabelField(dtype=torch.float) 
fields = [('clean_tweet',text_field),('label',label_field)] 
train, val, test = TabularDataset.splits(path=OUT_PATH, train='hate_train.csv',validation='hate_val.csv',
                                         test='hate_test.csv', 
                                         format='csv',skip_header=True,fields=fields)

In [ ]:
print(f'Type of "train:" {type(train)}\n Length of "train": {len(train)}\n' )
i = random.randint(0,len(train)) # generate a random index  within the lenth of train
print(f'Keys at index {i} of "train": {train[i].__dict__.keys()}\n')
print("Contents at random index:\n",vars(train.examples[i])) 

In [ ]:
vars(train)